Inainte sa trimiti notebook-ul:

* asigura-te ca totul se executa cum trebuie. Prima oara, **restarteaza kernel-ul** (in meniu, selecteaza Kernel$\rightarrow$Restart & Run all);

* nu lasa niciun mesaj la output (`disp`, `printf`, linii care nu sunt incheiate cu `;`) in functiile pe care le scrii decat daca ti se specifica explicit sa faci acest lucru! Orice abatere va fi **depunctata automat** de grader;

* citeste instructiunile de [aici](https://github.com/radusqrt/numerical-methods/blob/master/lab_instructions.md) cu atentie;

* asigura-te ca rezolvi toate celulele in care scrie `YOUR CODE HERE` sau `YOUR ANSWER HERE` si ca iti treci numele mai jos:

In [1]:
STUDENT_NAME = "Petruc_Rares";

---

# Laborator 4: Eliminari gaussiene
### Autor: Radu Stochitoiu

## Functii ajutatoare    

In [13]:
function [res] = isequal(A, B, err)
    res = norm(A - B) < err;
endfunction

function [x] = SST(A, b)
    n = length(b);
    x = [b(n) / A(n, n)];
    for i = n-1:-1:1
        x = [(b(i) - A(i, [i+1:n]) * x) / A(i,i); x];
    endfor
endfunction

## Problema 1

Implementati functia ```G(A, b)``` ce primeste ca parametri o matrice $A$ patratica, de lungime pozitiva, si un vector coloana $b$ si returneaza matricea superior triunghiulara rezultata $R$ si solutia sistemului $Ax=b$ aflata prin eliminare gaussiana.

In [14]:
function [x] = SST(A, b)
n = length(b);
x = zeros(n,1);
for i = [n:-1:1]
    x(i, 1) = (b(i)-A(i, [1:n])*x)/A(i, i);
endfor
endfunction

function [R, x] = G(A, b)
    n = size(A)(1);
    R = A;
    x = b;
    for j = 1:n-1
        for i = j+1:n
            x(i, 1) = x(i, 1) - R(i, j)/R(j, j) * x(j, 1);
            R(i, :) = R(i, :) - R(i, j)/R(j, j) * R(j, :);
        endfor
    endfor
    x = SST(R, x);
endfunction

In [15]:
% Test 1
A = [2 1 7; 3 1 0; 5 1 4];
b = [3 2 4]';
R_ref = [2 1 7; 0 -0.5 -10.5; 0 0 18];
[R, x] = G(A, b);
assert(isequal(R, R_ref, 1e-5));
assert(isequal(x, A\b, 1e-5));

## Problema 2

Implementati functia ```GPP(A, b)``` ce primeste ca parametri o matrice $A$ patratica, de lungime pozitiva, si un vector coloana $b$ si returneaza matricea superior triunghiulara rezultata $R$ si solutia sistemului $Ax=b$ aflata prin eliminare gaussiana cu pivotare partiala.

In [16]:
function [R, x] = GPP(A, b)
    n = size(A);
    x = b;
    R = A;
    for k = 1:n
        %partea de interschimbare
        for i = k:n
            for j = i+1:n
                if (abs(R(i, k)) < abs(R(j, k)))
                    aux = R(i, :);
                    R(i, :) = R(j, :);
                    R(j, :) = aux;
                    aux2 = x(i, 1);
                    x(i, 1) = x(j, 1);
                    x(j, 1) = aux2;
                endif
            endfor
        endfor     
        for p = k+1:n
            x(p, 1) = x(p, 1) - R(p, k)/R(k, k) * x(k, 1);
            R(p, :) = R(p, :) - R(p, k)/R(k, k) * R(k, :);            
        endfor
    endfor
    x = SST(R, x);
endfunction

In [17]:
% Test 1
A = [0 1 3; 2 1 3; -3 1 1];
b = [3 1 1]';
R_ref = [-3 1 1; 0 5/3 11/3; 0 0 4/5];
[R, x] = GPP(A, b);
assert(isequal(R, R_ref, 1e-5));
assert(isequal(x, A\b, 1e-5));

## Problema 3

Implementati functia ```GPPS(A, b)``` ce primeste ca parametri o matrice $A$ patratica, de lungime pozitiva, si un vector coloana $b$ si returneaza matricea superior triunghiulara rezultata $R$ si solutia sistemului $Ax=b$ aflata prin eliminare gaussiana cu pivotare partiala scalata.

In [18]:
function [x] = SST(A, b)
n = length(b);
x = zeros(n,1);
for i = [n:-1:1]
    x(i, 1) = (b(i)-A(i, [1:n])*x)/A(i, i);
endfor
endfunction

function [R, x] = GPPS(A, b)
    n = size(A)(1);
    R = A;
    x = b;
    for j = 1:n-1
        maxRowValues = zeros(n:1);
        for i=j:n
            maxRowValues(i, 1) = max(abs(R(i, :)));
        endfor
        array = zeros(n:1);
        % construiesc vectorul ajutator cu valorile ce trebuie comparate
        for i = j:n
            array(i, 1) = abs(R(i, j))/ maxRowValues(i, 1);
        endfor
        maxindex = 1;
        maxvalue = array(1, 1);
        for p = j:n
            if (array(p, 1) > maxvalue)
                maxvalue = array(p, 1);
                maxindex = p;
            endif
        endfor
        aux = x(maxindex, 1);
        x(maxindex, :) = x(j, 1);
        x(j, 1) = aux;
        aux2 = R(maxindex, :);
        R(maxindex, :) = R(j, :);
        R(j, :) = aux2;
        for p = j+1:n
            x(p, 1) = x(p, 1) - R(p, j)/R(j, j) * x(j, 1);
            R(p, :) = R(p, :) - R(p, j)/R(j, j) * R(j, :);            
        endfor
    endfor
    x = SST(R, x);
endfunction

In [19]:
% Test 1
A = [3 -13 9 3; -6 4 1 -18; 6 -2 2 4; 12 -8 6 10];
b = [-19 -34 16 26]';
R_ref = [6 -2 2 4; 0 -12 8 1; 0 0 -2/3 5/3; 0 0 0 -3];
[R, x] = GPPS(A, b);
assert(isequal(R, R_ref, 1e-5));
assert(isequal(x, A\b, 1e-5));

## Problema 4

Implementati functia ```GPT(A, b)``` ce primeste ca parametri o matrice $A$ patratica, de lungime pozitiva, si un vector coloana $b$ si returneaza matricea superior triunghiulara rezultata $R$ si solutia sistemului $Ax=b$ aflata prin eliminare gaussiana cu pivotare totala (se pastreaza ordinea initiala a elementelor lui $x$).

In [20]:
function [R, x] = GPT(A, b)
    R = A;
    x = b;
    n = size(A)(1);
    position = [1: 3];
    for j = 1:n
    maxval = R(j, j);
    maxi = j;
    maxj = j;
        for p = j:n
                for k = j:n
                    if (R(p, k) > maxval)
                        maxval = R(p, k);
                        maxi = p;
                        maxj = k;
                    endif
                endfor
        endfor
        % interschimbare linia j cu linia maxi
        aux = R(j, :);
        R(j, :) = R(maxi, :);
        R(maxi, :) = aux;
        % interschimbare coloana j cu, coloana maxj
        aux2 = R(:, j);
        R(:, j) = R(:, maxj);
        R(:, maxj) = aux2;
        % interschimbare coloana j cu, coloana maxj in vectorul pozitiilor
        aux3 = position(1, j);
        position(1, j) = position(1, maxj);
        position(1, maxj) = aux3;
        % interschimbare linia j cu lnia maxi in vectorul rezultat
        aux4 = x(j, :);
        x(j, :) = x(maxi, :);
        x(maxi, :) = aux4;
        for p = j+1:n
            x(p, 1) = x(p, 1) - R(p, j)/R(j, j) * x(j, 1);
            R(p, :) = R(p, :) - R(p, j)/R(j, j) * R(j, :);            
        endfor
    endfor
        aux = SST(R, x);
        for i = 1:n
            x(position(i), 1) = aux(i, 1);
        endfor
endfunction

In [21]:
% Test 1
A = [2 1 7; 3 1 0; 5 1 4];
b = [3 2 4]';
R_ref = [7 2 1; 0 27/7 3/7; 0 0 2/3];
[R, x] = GPT(A, b);
assert(isequal(R, R_ref, 1e-5));
assert(isequal(x, A\b, 1e-5));

## Problema 5 (BONUS)

Implementati functia ```GaussJordan(A)``` ce primeste ca parametru o matrice $A$ patratica, de lungime pozitiva, si returneaza inversa lui $A$ calculata prin procedeul Gauss-Jordan.

In [22]:
function [R, x] = GPPS_(A, b)
    n = size(A)(1);
    R = A;
    x = b;
    maxRowValues = zeros(n:1);
    for i=1:n
        maxRowValues(i, 1) = max(abs(R(i, :)));
    endfor
    for j = 1:n-1
        array = zeros(n:1);
        % construiesc vectorul ajutator cu valorile ce trebuie comparate
        for i = j:n
            array(i, 1) = abs(R(i, j))/ maxRowValues(i, 1);
        endfor
        maxindex = 1;
        maxvalue = array(1, 1);
        for p = j:n
            if (array(p, 1) > maxvalue)
                maxvalue = array(p, 1);
                maxindex = p;
            endif
        endfor
        aux = x(maxindex, :);
        x(maxindex, :) = x(j, :);
        x(j, :) = aux;
        aux2 = R(maxindex, :);
        R(maxindex, :) = R(j, :);
        R(j, :) = aux2;
        for p = j+1:n
            x(p, :) = x(p, :) - R(p, j)/R(j, j) * x(j, :);
            R(p, :) = R(p, :) - R(p, j)/R(j, j) * R(j, :);            
        endfor
    endfor
endfunction

function [R, x] = GPPS__(A, b)
    n = size(A)(1);
    R = A;
    x = b;
    maxRowValues = zeros(n:1);
    for i=n:-1:1
        maxRowValues(i, 1) = max(abs(R(i, :)));
    endfor
    for j = n:-1:2
        array = zeros(3:1);
        % construiesc vectorul ajutator cu valorile ce trebuie comparate
        for i = j:-1:1
        array(i, 1) = abs(R(i, j))/ maxRowValues(i, 1);
        endfor
        maxindex = 1;
        maxvalue = array(1, 1);
        for p = j:-1:1
            if (array(p, 1) > maxvalue)
                maxvalue = array(p, 1);
                maxindex = p;
            endif
        endfor
        % if maxvalue == 0 return;
        aux = x(maxindex, :);
        x(maxindex, :) = x(j, :);
        x(j, :) = aux;
        aux2 = R(maxindex, :);
        R(maxindex, :) = R(j, :);
        R(j, :) = aux2;
        for p = j-1:-1:1
            x(p, :) = x(p, :) - R(p, j)/R(j, j) * x(j, :);
            R(p, :) = R(p, :) - R(p, j)/R(j, j) * R(j, :);            
        endfor
    endfor
endfunction

function [x] = scale_them(I3, invA)
    n = size(I3)(1);
    x = invA;
    for i = 1:n
        x(i, :) = x(i, :)  / I3(i, i);
    endfor
endfunction

function [invA] = GaussJordan(A)
    n = size(A)(1);
    auxA = A;
    array = zeros(n);
    invA = eye(n);
    % make it LU
    [I3, invA]  = GPPS_(A, invA);
    %make it RU
    [I3, invA]  = GPPS__(I3, invA);
    %Make I3 identity matrix
    [invA] = scale_them(I3, invA);
endfunction

In [23]:
% Test 1
A = [2 3 0; 1 -2 -1; 2 0 -1];
[invA] = GaussJordan(A);
assert(isequal(invA, inv(A), 1e-5));